In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import warnings
import time
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

warnings.filterwarnings('ignore')

In [2]:
train  = pd.read_csv('competition_data/train.csv')
test  = pd.read_csv('competition_data/test.csv')
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_columns', 100)

In [3]:
train

,index,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,Q11,Q12,Q13,Q14,Q15,Q16,Q17,Q18,Q19,Q20,Q21,Q22,Q23,Q24,Q25,Q26,country,introelapse,testelapse,surveyelapse,TIPI1,TIPI2,TIPI3,TIPI4,TIPI5,TIPI6,TIPI7,TIPI8,TIPI9,TIPI10,VCL1,VCL2,VCL3,VCL4,VCL5,VCL6,VCL7,VCL8,VCL9,VCL10,VCL11,VCL12,VCL13,VCL14,VCL15,VCL16,education,urban,gender,engnat,age,hand,religion,orientation,voted,married,familysize,ASD,nerdiness
0,0,1.0,5.0,5.0,5.0,1.0,4.0,5.0,5.0,1.0,3.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,1.0,5.0,5.0,1.0,5.0,1.0,5.0,1.0,1.0,USA,3,553,6,4.0,3.0,5.0,1.0,3.0,5.0,5.0,3.0,5.0,3.0,1,1,0,1,1,0,0,0,0,1,0,0,0,1,1,1,2.0,1,3.0,1.0,20,2.0,12.0,4.0,2.0,1.0,4.0,2.0,1
1,1,4.0,4.0,4.0,4.0,4.0,5.0,4.0,4.0,3.0,3.0,1.0,4.0,5.0,3.0,1.0,2.0,4.0,5.0,1.0,3.0,1.0,1.0,5.0,3.0,2.0,5.0,USA,5,85,120,4.0,2.0,3.0,5.0,3.0,2.0,5.0,1.0,2.0,2.0,1,1,1,1,1,0,1,0,0,1,0,0,1,1,1,1,4.0,2,2.0,1.0,49,1.0,2.0,1.0,1.0,2.0,4.0,2.0,1
2,2,4.0,5.0,5.0,4.0,3.0,5.0,5.0,5.0,4.0,4.0,2.0,5.0,5.0,5.0,1.0,3.0,5.0,3.0,5.0,2.0,2.0,1.0,2.0,4.0,2.0,5.0,NLD,9,108,100,1.0,2.0,3.0,1.0,5.0,5.0,3.0,4.0,5.0,2.0,1,1,0,1,1,0,1,1,0,1,0,0,1,1,1,1,2.0,1,1.0,2.0,43,1.0,2.0,2.0,2.0,3.0,4.0,2.0,1
3,3,4.0,4.0,4.0,2.0,4.0,3.0,3.0,5.0,3.0,4.0,5.0,2.0,2.0,4.0,4.0,2.0,4.0,5.0,4.0,3.0,3.0,4.0,3.0,4.0,4.0,2.0,USA,2,121,139,3.0,3.0,3.0,4.0,5.0,3.0,4.0,4.0,3.0,3.0,1,1,0,1,1,0,0,0,0,1,0,0,1,1,1,1,1.0,3,1.0,1.0,17,2.0,1.0,1.0,2.0,1.0,2.0,2.0,1
4,4,4.0,4.0,4.0,4.0,3.0,3.0,4.0,2.0,3.0,4.0,4.0,4.0,3.0,5.0,5.0,2.0,4.0,1.0,4.0,2.0,4.0,2.0,3.0,4.0,4.0,4.0,ITA,3,640,216,3.0,3.0,4.0,4.0,4.0,4.0,3.0,4.0,3.0,2.0,1,1,0,1,1,0,0,1,0,1,0,0,0,1,0,1,1.0,2,2.0,2.0,18,2.0,12.0,1.0,2.0,1.0,1.0,2.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,14995,2.0,5.0,4.0,3.0,3.0,4.0,4.0,4.0,3.0,4.0,1.0,4.0,4.0,3.0,4.0,4.0,2.0,5.0,2.0,4.0,1.0,2.0,5.0,4.0,2.0,4.0,USA,12,103,161,2.0,2.0,4.0,3.0,3.0,5.0,3.0,3.0,3.0,3.0,1,1,1,1,1,0,0,0,0,1,0,1,1,1,1,1,2.0,2,2.0,1.0,17,1.0,1.0,3.0,2.0,1.0,3.0,2.0,0
14996,14996,5.0,4.0,5.0,4.0,4.0,5.0,5.0,4.0,4.0,5.0,1.0,4.0,4.0,4.0,2.0,5.0,5.0,4.0,1.0,5.0,3.0,4.0,4.0,5.0,4.0,5.0,USA,31,106,179,3.0,2.0,4.0,5.0,4.0,3.0,4.0,1.0,2.0,2.0,1,1,0,1,1,0,0,0,0,1,0,0,0,1,1,1,4.0,1,2.0,2.0,45,1.0,3.0,1.0,1.0,2.0,3.0,2.0,1
14997,14997,4.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,4.0,5.0,4.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,4.0,5.0,5.0,2.0,5.0,5.0,3.0,4.0,USA,17,103,168,1.0,3.0,2.0,5.0,1.0,5.0,3.0,3.0,1.0,1.0,1,1,0,1,1,0,0,0,0,1,0,0,1,1,1,1,2.0,2,2.0,1.0,20,1.0,1.0,2.0,1.0,1.0,3.0,1.0,1
14998,14998,5.0,5.0,4.0,5.0,5.0,5.0,5.0,1.0,5.0,5.0,3.0,5.0,4.0,4.0,1.0,5.0,4.0,5.0,5.0,2.0,5.0,3.0,5.0,3.0,3.0,5.0,USA,14,68,109,1.0,1.0,3.0,5.0,4.0,5.0,5.0,4.0,2.0,1.0,1,1,1,1,1,0,0,1,0,1,0,1,1,1,1,1,3.0,2,2.0,1.0,29,1.0,12.0,4.0,2.0,2.0,2.0,1.0,0


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 70 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   index         15000 non-null  int64  
 1   Q1            14959 non-null  float64
 2   Q2            14931 non-null  float64
 3   Q3            14950 non-null  float64
 4   Q4            14929 non-null  float64
 5   Q5            14962 non-null  float64
 6   Q6            14952 non-null  float64
 7   Q7            14924 non-null  float64
 8   Q8            14952 non-null  float64
 9   Q9            14944 non-null  float64
 10  Q10           14928 non-null  float64
 11  Q11           14941 non-null  float64
 12  Q12           14933 non-null  float64
 13  Q13           14960 non-null  float64
 14  Q14           14964 non-null  float64
 15  Q15           14955 non-null  float64
 16  Q16           14967 non-null  float64
 17  Q17           14963 non-null  float64
 18  Q18           14937 non-nu

In [5]:
train = train.drop(['index', 'country'],axis = 1)
test =test.drop(['index', 'country'],axis = 1)

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 68 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Q1            14959 non-null  float64
 1   Q2            14931 non-null  float64
 2   Q3            14950 non-null  float64
 3   Q4            14929 non-null  float64
 4   Q5            14962 non-null  float64
 5   Q6            14952 non-null  float64
 6   Q7            14924 non-null  float64
 7   Q8            14952 non-null  float64
 8   Q9            14944 non-null  float64
 9   Q10           14928 non-null  float64
 10  Q11           14941 non-null  float64
 11  Q12           14933 non-null  float64
 12  Q13           14960 non-null  float64
 13  Q14           14964 non-null  float64
 14  Q15           14955 non-null  float64
 15  Q16           14967 non-null  float64
 16  Q17           14963 non-null  float64
 17  Q18           14937 non-null  float64
 18  Q19           14947 non-nu

In [7]:
train.isna().sum()

Q1               41
Q2               69
Q3               50
Q4               71
Q5               38
Q6               48
Q7               76
Q8               48
Q9               56
Q10              72
Q11              59
Q12              67
Q13              40
Q14              36
Q15              45
Q16              33
Q17              37
Q18              63
Q19              53
Q20              45
Q21              39
Q22              38
Q23              50
Q24              61
Q25              44
Q26              68
introelapse       0
testelapse        0
surveyelapse      0
TIPI1            53
TIPI2            66
TIPI3            79
TIPI4            64
TIPI5            70
TIPI6            62
TIPI7            64
TIPI8            65
TIPI9            64
TIPI10           80
VCL1              0
VCL2              0
VCL3              0
VCL4              0
VCL5              0
VCL6              0
VCL7              0
VCL8              0
VCL9              0
VCL10             0
VCL11             0


In [8]:
#train을 target과 feature로 나눠줍니다.
#train_ = train.dropna(axis=0)
train_ = train

In [9]:
train_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 68 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Q1            14959 non-null  float64
 1   Q2            14931 non-null  float64
 2   Q3            14950 non-null  float64
 3   Q4            14929 non-null  float64
 4   Q5            14962 non-null  float64
 5   Q6            14952 non-null  float64
 6   Q7            14924 non-null  float64
 7   Q8            14952 non-null  float64
 8   Q9            14944 non-null  float64
 9   Q10           14928 non-null  float64
 10  Q11           14941 non-null  float64
 11  Q12           14933 non-null  float64
 12  Q13           14960 non-null  float64
 13  Q14           14964 non-null  float64
 14  Q15           14955 non-null  float64
 15  Q16           14967 non-null  float64
 16  Q17           14963 non-null  float64
 17  Q18           14937 non-null  float64
 18  Q19           14947 non-nu

In [10]:
train_S = StandardScaler().fit_transform(train_.iloc[:, :-1])
train_S = pd.DataFrame(train_S)
train_S

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66
0,-2.712765,0.798789,0.759494,0.987378,-2.389216,0.276388,0.770711,0.860949,-2.608670,-1.060153,1.195496,0.973683,0.982827,1.100144,1.306409,1.119291,0.931739,-2.195271,1.043924,1.131658,-1.222889,1.880924,-2.084149,0.764801,-1.562594,-3.069350,-0.055230,0.019387,-0.015577,1.196466,-0.035456,1.535004,-1.934982,-0.960823,1.189100,1.448234,-0.034278,1.722858,0.795350,0.207643,0.291532,-1.060068,0.180641,0.235785,-0.336166,-0.555788,-0.880306,-0.268405,0.214365,-0.623713,-0.515994,-1.785059,0.321728,0.204478,0.140655,-0.341126,-1.598170,2.344213,-0.727388,-0.021261,1.690718,2.306661,1.759429,0.761234,-0.386113,0.052094,0.255538
1,0.024766,-0.056173,-0.217736,0.167638,0.120023,1.130398,-0.172934,0.092921,-0.819101,-1.060153,-1.352747,0.163068,0.982827,-0.623683,-1.334308,-1.024425,0.051809,0.829079,-1.542869,-0.575245,-1.222889,-1.027174,0.855981,-1.410215,-0.836294,0.814102,-0.055086,-0.035590,-0.014939,1.196466,-0.979300,-0.466464,1.466724,-0.960823,-1.518091,1.448234,-1.787506,-1.088322,-0.171538,0.207643,0.291532,0.943336,0.180641,0.235785,-0.336166,1.799247,-0.880306,-0.268405,0.214365,-0.623713,-0.515994,0.560206,0.321728,0.204478,0.140655,1.830047,-0.225014,0.536768,-0.727388,0.070207,-0.384071,-0.612550,-0.693256,-1.313657,1.759952,0.052094,0.255538
2,0.024766,0.798789,0.759494,0.167638,-0.716390,1.130398,0.770711,0.860949,0.075683,-0.105574,-0.715686,0.973683,0.982827,1.100144,-1.334308,-0.309853,0.931739,-0.683096,1.043924,-1.428697,-0.601623,-1.027174,-1.349116,-0.322707,-0.836294,0.814102,-0.054797,-0.032888,-0.015051,-1.410469,-0.979300,-0.466464,-1.934982,1.232616,1.189100,-0.521069,0.842336,1.722858,-0.171538,0.207643,0.291532,-1.060068,0.180641,0.235785,-0.336166,1.799247,1.135969,-0.268405,0.214365,-0.623713,-0.515994,0.560206,0.321728,0.204478,0.140655,-0.341126,-1.598170,-1.270677,1.374783,0.051283,-0.384071,-0.612550,0.124306,0.761234,3.906017,0.052094,0.255538
3,0.024766,-0.056173,-0.217736,-1.471842,0.120023,-0.577622,-1.116579,0.860949,-0.819101,-0.105574,1.195496,-1.458163,-1.287686,0.238230,0.646230,-1.024425,0.051809,0.829079,0.397226,-0.575245,0.019642,1.153900,-0.614084,-0.322707,0.616306,-2.098487,-0.055303,-0.031361,-0.014833,0.327487,-0.035456,-0.466464,0.616297,1.232616,-0.615694,0.463582,0.842336,-0.151262,0.795350,0.207643,0.291532,-1.060068,0.180641,0.235785,-0.336166,-0.555788,-0.880306,-0.268405,0.214365,-0.623713,-0.515994,0.560206,0.321728,0.204478,0.140655,-1.426712,1.148141,-1.270677,-0.727388,-0.030724,1.690718,-0.904471,-0.693256,0.761234,-0.386113,-0.030864,0.255538
4,0.024766,-0.056173,-0.217736,0.167638,-0.716390,-0.577622,-0.172934,-1.443133,-0.819101,-0.105574,0.558436,0.163068,-0.530849,1.100144,1.306409,-1.024425,0.051809,-2.195271,0.397226,-1.428697,0.640907,-0.300149,-0.614084,-0.322707,0.616306,-0.156761,-0.055230,0.029607,-0.014402,0.327487,-0.035456,0.534270,0.616297,0.135896,0.286703,-0.521069,0.842336,-0.151262,-0.171538,0.207643,0.291532,-1.060068,0.180641,0.235785,-0.336166,-0.555788,1.135969,-0.268405,0.214365,-0.623713,-0.515994,-1.785059,0.321728,-4.890495,0.140655,-1.426712,-0.225014,0.536768,1.374783,-0.027570,1.690718,2.306661,-0.693256,0.761234,-0.386113,-0.072343,0.255538
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,-1.800255,0.798789,-0.217736,-0.652102,-0.716390,0.276388,-0.172934,0.092921,-0.819101,-0.105574,-1.352747,0.163068,0.225989,-0.623683,0.646230,0.404719,-1.708051,0.829079,-0.896170,0.278207,-1.222889,-0.300149,0.855981,-0.322707,-0.836294,-0.156761,-0.054580,-0.033475,-0.014709,-0.541491,-0.979300,0.534270,-0.234129,-0.960823,1.189100,

In [11]:
train_S.columns=['Q1','Q2','Q3','Q4','Q5','Q6','Q7','Q8'
               ,'Q9','Q10','Q11','Q12','Q13','Q14','Q15','Q16'
               ,'Q17','Q18','Q19','Q20','Q21','Q22','Q23','Q24'
               ,'Q25','Q26','introelapse','testelapse','surveyelapse'
                ,'TIPI1','TIPI2','TIPI3'
               ,'TIPI4','TIPI5','TIPI6','TIPI7','TIPI8','TIPI9','TIPI10'
                ,'VCL1','VCL2','VCL3','VCL4','VCL5','VCL6','VCL7'
               ,'VCL8','VCL9','VCL10','VCL11','VCL12','VCL13','VCL14'
               ,'VCL15','VCL16','education','urban','gender','engnat','age'
               ,'hand','religion','orientation','voted','married','familysize'
                ,'ASD']

In [12]:
train_S

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,Q11,Q12,Q13,Q14,Q15,Q16,Q17,Q18,Q19,Q20,Q21,Q22,Q23,Q24,Q25,Q26,introelapse,testelapse,surveyelapse,TIPI1,TIPI2,TIPI3,TIPI4,TIPI5,TIPI6,TIPI7,TIPI8,TIPI9,TIPI10,VCL1,VCL2,VCL3,VCL4,VCL5,VCL6,VCL7,VCL8,VCL9,VCL10,VCL11,VCL12,VCL13,VCL14,VCL15,VCL16,education,urban,gender,engnat,age,hand,religion,orientation,voted,married,familysize,ASD
0,-2.712765,0.798789,0.759494,0.987378,-2.389216,0.276388,0.770711,0.860949,-2.608670,-1.060153,1.195496,0.973683,0.982827,1.100144,1.306409,1.119291,0.931739,-2.195271,1.043924,1.131658,-1.222889,1.880924,-2.084149,0.764801,-1.562594,-3.069350,-0.055230,0.019387,-0.015577,1.196466,-0.035456,1.535004,-1.934982,-0.960823,1.189100,1.448234,-0.034278,1.722858,0.795350,0.207643,0.291532,-1.060068,0.180641,0.235785,-0.336166,-0.555788,-0.880306,-0.268405,0.214365,-0.623713,-0.515994,-1.785059,0.321728,0.204478,0.140655,-0.341126,-1.598170,2.344213,-0.727388,-0.021261,1.690718,2.306661,1.759429,0.761234,-0.386113,0.052094,0.255538
1,0.024766,-0.056173,-0.217736,0.167638,0.120023,1.130398,-0.172934,0.092921,-0.819101,-1.060153,-1.352747,0.163068,0.982827,-0.623683,-1.334308,-1.024425,0.051809,0.829079,-1.542869,-0.575245,-1.222889,-1.027174,0.855981,-1.410215,-0.836294,0.814102,-0.055086,-0.035590,-0.014939,1.196466,-0.979300,-0.466464,1.466724,-0.960823,-1.518091,1.448234,-1.787506,-1.088322,-0.171538,0.207643,0.291532,0.943336,0.180641,0.235785,-0.336166,1.799247,-0.880306,-0.268405,0.214365,-0.623713,-0.515994,0.560206,0.321728,0.204478,0.140655,1.830047,-0.225014,0.536768,-0.727388,0.070207,-0.384071,-0.612550,-0.693256,-1.313657,1.759952,0.052094,0.255538
2,0.024766,0.798789,0.759494,0.167638,-0.716390,1.130398,0.770711,0.860949,0.075683,-0.105574,-0.715686,0.973683,0.982827,1.100144,-1.334308,-0.309853,0.931739,-0.683096,1.043924,-1.428697,-0.601623,-1.027174,-1.349116,-0.322707,-0.836294,0.814102,-0.054797,-0.032888,-0.015051,-1.410469,-0.979300,-0.466464,-1.934982,1.232616,1.189100,-0.521069,0.842336,1.722858,-0.171538,0.207643,0.291532,-1.060068,0.180641,0.235785,-0.336166,1.799247,1.135969,-0.268405,0.214365,-0.623713,-0.515994,0.560206,0.321728,0.204478,0.140655,-0.341126,-1.598170,-1.270677,1.374783,0.051283,-0.384071,-0.612550,0.124306,0.761234,3.906017,0.052094,0.255538
3,0.024766,-0.056173,-0.217736,-1.471842,0.120023,-0.577622,-1.116579,0.860949,-0.819101,-0.105574,1.195496,-1.458163,-1.287686,0.238230,0.646230,-1.024425,0.051809,0.829079,0.397226,-0.575245,0.019642,1.153900,-0.614084,-0.322707,0.616306,-2.098487,-0.055303,-0.031361,-0.014833,0.327487,-0.035456,-0.466464,0.616297,1.232616,-0.615694,0.463582,0.842336,-0.151262,0.795350,0.207643,0.291532,-1.060068,0.180641,0.235785,-0.336166,-0.555788,-0.880306,-0.268405,0.214365,-0.623713,-0.515994,0.560206,0.321728,0.204478,0.140655,-1.426712,1.148141,-1.270677,-0.727388,-0.030724,1.690718,-0.904471,-0.693256,0.761234,-0.386113,-0.030864,0.255538
4,0.024766,-0.056173,-0.217736,0.167638,-0.716390,-0.577622,-0.172934,-1.443133,-0.819101,-0.105574,0.558436,0.163068,-0.530849,1.100144,1.306409,-1.024425,0.051809,-2.195271,0.397226,-1.428697,0.640907,-0.300149,-0.614084,-0.322707,0.616306,-0.156761,-0.055230,0.029607,-0.014402,0.327487,-0.035456,0.534270,0.616297,0.135896,0.286703,-0.521069,0.842336,-0.151262,-0.171538,0.207643,0.291532,-1.060068,0.180641,0.235785,-0.336166,-0.555788,1.135969,-0.268405,0.214365,-0.623713,-0.515994,-1.785059,0.321728,-4.890495,0.140655,-1.426712,-0.225014,0.536768,1.374783,-0.027570,1.690718,2.306661,-0.693256,0.761234,-0.386113,-0.072343,0.255538
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,-1.800255,0.798789,-0.217736,-0.652102,-0.716390,0.276388,-0.172934,0.092921,-0.819101,-0.105574,-1.352747,0.163068,0.225989,-0.623683,0.646230,0.404719,-1.70805

In [13]:
train_S['nerdiness']=train_['nerdiness']

In [14]:
train_.iloc[:, -1]=train_['nerdiness']

In [15]:
train_x = train_S.drop(['nerdiness'],axis=1)
train_y = train_S['nerdiness']

In [16]:
start_time=time.time()
lgb= lgb.LGBMClassifier(
            n_estimators=1000
        )
parametes= {
    'num_leaves':[20,40,60,80,100],'min_child_samples':[5,10,15],
    'max_depth':[-1,5,10,20],'learning_rate':[0.05,0.1,0.2],'reg_alpha':[0,0.01,0.03]
}
clf=RandomizedSearchCV(lgb,parametes,scoring='roc_auc',n_iter=100)
clf.fit(X=train_x,y=train_y)
print(clf.best_params_)
end_time=time.time()
print('소요 시간 : ')
print(end_time-start_time)

{'reg_alpha': 0, 'num_leaves': 80, 'min_child_samples': 10, 'max_depth': 20, 'learning_rate': 0.1}
소요 시간 : 
1111.9319813251495


In [17]:
# 재학습 위한 공간

In [18]:
#lgbm_clf.fit(train_x, train_y)
lgbm_pred = clf.predict(test)

In [19]:
submission = pd.read_csv('competition_data/sample_submission.csv')

submission

,index,nerdiness
0,0,-1
1,1,-1
2,2,-1
3,3,-1
4,4,-1
...,...,...
35447,35447,-1
35448,35448,-1
35449,35449,-1
35450,35450,-1


In [20]:
submission["nerdiness"] = lgbm_pred

In [21]:
submission

,index,nerdiness
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1
...,...,...
35447,35447,1
35448,35448,1
35449,35449,1
35450,35450,1


In [22]:
submission.to_csv("baseline_withNA-withStd-Random1000.csv", index = False)